In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (6,6)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from keras.layers import Dense, Input, Activation, Conv1D
from keras.layers import Dropout, MaxPooling1D, Flatten, Concatenate, Reshape
from keras.models import Sequential, Model, load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import np_utils

import re
import random
import os

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Amaan_Hussain\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Using TensorFlow backend.


In [2]:
def load_as_list(fname):
    df = pd.read_csv(fname)
    id = df['id'].values.tolist()
    label = df['label'].values.tolist()
    tweets = df['tweet'].values.tolist()
    return tweets, label

In [6]:
# getting the tweets and labels in a pandas dataframe
tweets, label = load_as_list("train.csv")
# downloading the stopwords from nltk library
stop_words = stopwords.words('english')
# print(stop_words)

print(len(tweets))

# manually seperating 80% of the train data
x_train = tweets[:25570]

#removing all punctuations from the train data
for i in range(len(x_train)):
    x_train[i] = re.sub(r'[^\w\s]', '', x_train[i])
    

# print(x_train[:15])
print(len(x_train))



31962
25570


In [35]:
# removing stop_words from tf_idf and setting min occurences of a word to 30
tfidf = TfidfVectorizer(stop_words=stop_words,min_df=30)

x = tfidf.fit_transform(x_train).toarray()
y_train = np.array(label[:25570])

y_train = np.reshape(y_train, (-1,1))

# print(type(x),x.shape)
# print(x[:2])
tfidf_sentence_value = []

for i in range(len(x)):
    count = 0
    num_count = len([ele for ele in x[i] if ele > 0])
    if num_count > 0:
        tfidf_sentence_value.append([sum(x[i]) / num_count]) 
    else:
        tfidf_sentence_value.append([0])
        

# print(len(tfidf_sentence_value))
# tfidf_values = np.array(tfidf_sentence_value)
# print(tfidf_values.shape)
# y_train = np.array(label[:25570])
# print(y_train[:5])
# tfidf_values.reshape(-1)
# print(tfidf_values.shape)
# print(tfidf_values[:5])

print(f"TFIDF Transform Shape: {x.shape} with length: {len(x)}")
print(f"Sample: \n{x[:5]}\n\n")
print(f"TFIDF Value per Sentence Shape: {tfidf_values.shape} with length: {len(tfidf_values)}")
print(f"Labels Shape: {y_train.shape} with length: {len(y_train)}")
    

TFIDF Transform Shape: (25570, 1005) with length: 25570
TFIDF Value per Sentence Shape: (25570, 1) with length: 25570
Labels Shape: (25570, 1) with length: 25570


In [34]:
model = Sequential([
    Dense(256, activation='relu', input_shape=(tfidf_values.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid'),
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(tfidf_values,y_train,epochs=50,batch_size=1000)

with open('modelsummary.txt', 'w') as f:
    model.summary(print_fn=lambda x: f.write(x + '\n'))

ERROR! Session/line number was not unique in database. History logging moved to new session 133
Epoch 1/50
25570/25570 [==============================] - 0s 9us/step - loss: 0.4643 - accuracy: 0.8974
Epoch 2/50
25570/25570 [==============================] - 0s 3us/step - loss: 0.2772 - accuracy: 0.9296
Epoch 3/50
25570/25570 [==============================] - 0s 4us/step - loss: 0.2692 - accuracy: 0.9296
Epoch 4/50
25570/25570 [==============================] - 0s 3us/step - loss: 0.2665 - accuracy: 0.9296
Epoch 5/50
25570/25570 [==============================] - 0s 4us/step - loss: 0.2642 - accuracy: 0.9296
Epoch 6/50
25570/25570 [==============================] - 0s 4us/step - loss: 0.2620 - accuracy: 0.9296
Epoch 7/50
25570/25570 [==============================] - 0s 3us/step - loss: 0.2596 - accuracy: 0.9296
Epoch 8/50
25570/25570 [==============================] - 0s 3us/step - loss: 0.2573 - accuracy: 0.9296
Epoch 9/50
25570/25570 [==============================] - 0s 4us/step - 

In [ ]:
pwd